In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance ta scikit-learn
# !pip install line_profiler


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#!git clone https://github.com/Klok-e/python-price-predict-experiment.git

import sys

sys.path.insert(0, 'python-price-predict-experiment')

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/computed-data/")
# except:
#   pass

In [3]:
from util import download_and_process_data_if_available

df_tickers = download_and_process_data_if_available("cache/df_tickers.pkl")

Loading data from cache


In [ ]:
%load_ext tensorboard
%tensorboard --logdir data/tensorboard
# %load_ext tensorboard
# %tensorboard --logdir /content/drive/MyDrive/colab/computed-data/tensorboard

In [ ]:
from train_model import train_model
from env import CustomEnv

from stable_baselines3.common.env_checker import check_env

env = CustomEnv(df_tickers)
check_env(env)
del env

hidden_size_list = [128, 256, 512]
lstm_layers_list = [1, 2]
arch_list = [[256, 256, 256]]
window_size_list = [32, 64, 128]
for hidden_size in hidden_size_list:
    for lstm_layers in lstm_layers_list:
        for arch in arch_list:
            for window_size in window_size_list:
                print(f"hidden_size {hidden_size}, lstm_layers {lstm_layers}, window_size {window_size}")
                # dir = "/content/drive/MyDrive/colab/computed-data"
                dir = "data"
                train_model(df_tickers, hidden_size, lstm_layers, arch, 2_500_000, window_size, 5, dir)

In [ ]:
# !zip -r /content/drive/MyDrive/colab/model/tensorboard.zip /content/tensorboard/
# !zip -r /content/drive/MyDrive/colab/model/rl-model.zip /content/rl-model/

In [ ]:
# evaluations = np.load("rl-model-best-model/results/evaluations.npz")
# evaluations_df = pd.DataFrame()
# evaluations_df["timesteps"] = evaluations["timesteps"]
# evaluations_df["results"] = evaluations["results"].mean(axis=1)
# evaluations_df